As far as I concerned, this game is a muti-classify problem.
I will solve in this way:

 1.  analyse train pictures and train.csv;
 2. use CNN and CV train model;
 3. pridict test labels.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


In [ ]:
for f in os.listdir('../input'):
    if not os.path.isdir('../input/' + f):
        print(f.ljust(30) + str(round(os.path.getsize('../input/' + f) / 1000000, 2)) + 'MB')
    else:
        sizes = [os.path.getsize('../input/'+f+'/'+x)/1000000 for x in os.listdir('../input/' + f)]
        print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + ' ({} files)'.format(len(sizes)))

In [ ]:
df_train = pd.read_csv('../input/train.csv')

df_train.head()

we can see the labels in  df_train, has **3 rules**:

 1. every label should has a atmospheric label, as 'haze ,'clear','cloudy','partly_cloudy ',if not ,this label would be a noise.

 2. when atmospheric label is 'cloudy', there is no other lable, if not, as a noise.

 3. besides atmospheric labels, the left labels has some relationship, such as 'road' and 'habitation'


First, let's deal with a original data, dorp out noise data.

In [ ]:
train_labels = df_train['tags'].apply(lambda x: x.split(' '))
train_length=len(train_labels)
train_labels
             

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

labels17 = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))
labels17





In [ ]:
atmos_label=['haze','clear','cloudy','partly_cloudy']
common_label=[]
for i in range(len(atmos_label)):
    for j in range(len(labels17)):
        if labels17[j]!=atmos_label[i]:
            common_label.append(labels17[j])
common_label=np.array(common_label)
common_label=np.unique(common_label)
common_label=list(common_label)
common_label
len(train_labels[1])

In [ ]:
valible_train_haze=[]
valible_train_clear=[]
valible_train_cloudy=[]
valible_train_partly_cloudy=[]
for i in range(train_length):
    for j in range(len(train_labels[i])):
        if train_labels[i][j] == atmos_label[0]:
            valible_train_haze.append(i)
        elif train_labels[i][j] == atmos_label[1]:
             valible_train_clear.append(i)
        elif train_labels[i][j] == atmos_label[2]:   
             valible_train_cloudy.append(i)
        elif train_labels[i][j] == atmos_label[3]: 
              valible_train_partly_cloudy.append(i)
print(len(valible_train_haze))
print(len(valible_train_clear))  
print(len(valible_train_cloudy))
print(len(valible_train_partly_cloudy))  

now we split train data into four parts according atmospheric label,


the number is HAZE:2695 CLEAR:28203 CLOUDY:2330 PARTLY_CLOUDY:7251.


if the the label is  'CLOUDY', there should common label, drop out noise. 

In [ ]:
valible_train_cloudy_dropout=[]
print(len(valible_train_cloudy))
for i in valible_train_cloudy:
    if(len(train_labels[i]))>1:
       continue
    else:
       valible_train_cloudy_dropout.append(i)
print(len(valible_train_cloudy_dropout))

In [ ]:
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

import cv2
from tqdm import tqdm

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score
import time

In [ ]:

from collections import Counter, defaultdict
counts = defaultdict(int)
for l in train_labels:
    for l2 in l:
        counts[l2] += 1
counts

In [ ]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/sample_submission.csv')


for f, tags in tqdm(df_train.values[0:10000], miniters=1000):
    img = cv2.imread('../input/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(4)
    for t in tags.split(' '):
            if t in atmos_label:
                num_index=atmos_label.index(t)
                targets[num_index] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
targets

for f, tags in tqdm(df_test.values[0:10000], miniters=1000):
    img = cv2.imread('../input/test-jpg-v2/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (32, 32)))
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

print(x_train.shape)
print(y_train.shape)

In [ ]:
from keras.layers import BatchNormalization

nfolds = 2

num_fold = 0
sum_score = 0

yfull_test = []
yfull_train =[]

kf = KFold(len(y_train), n_folds=nfolds, shuffle=True, random_state=1)

for train_index, test_index in kf:
        start_time_model_fitting = time.time()
        
        X_train = x_train[train_index]
        Y_train = y_train[train_index]
        X_valid = x_train[test_index]
        Y_valid = y_train[test_index]

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        kfold_weights_path = os.path.join('', 'weights_kfold_' + str(num_fold) + '.h5')
        
        model = Sequential()
        model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(32, 32, 3)))
        model.add(Conv2D(32, 3, 3, activation='relu'))
        BatchNormalization(axis=1)
        model.add(MaxPooling2D(pool_size=(2, 2)))
        BatchNormalization(axis=1)
        model.add(Conv2D(64, 3, 3, activation='relu'))
        model.add(Conv2D(64, 3, 3, activation='relu'))
        BatchNormalization(axis=1)
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        BatchNormalization()
        model.add(Dense(128, activation='relu'))
        BatchNormalization()
        model.add(Dropout(0.5))
        model.add(Dense(4, activation='softmax'))

        model.compile(loss='categorical_crossentropy', 
                      optimizer='adam',
                      metrics=['accuracy'])
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=2, verbose=0),
            ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)]
        
        model.fit(x = X_train, y= Y_train, validation_data=(X_valid, Y_valid),
                  batch_size=128,verbose=2, nb_epoch=7,callbacks=callbacks,
                  shuffle=True)
        
        if os.path.isfile(kfold_weights_path):
            model.load_weights(kfold_weights_path)
        
        p_valid = model.predict(X_valid, batch_size = 128, verbose=2)
        print(fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))
        
        p_test = model.predict(x_train, batch_size = 128, verbose=2)
        yfull_train.append(p_test)
        
        p_test = model.predict(x_test, batch_size = 128, verbose=2)
        yfull_test.append(p_test)
        

In [ ]:
yfull_train
result_haze=[]
result_clear=[]
result_cloudy=[]
result_partly_cloudy=[]



In [ ]:
yfull_test
result = np.array(yfull_test[0])
for i in range(1, nfolds):
    result += np.array(yfull_test[i])
result /= nfolds
result = pd.DataFrame(result, columns = atmos_label)
result

In [ ]:
from tqdm import tqdm

preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    print(a)
    a = a.loc[a[i] == True]
    print(a)
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

In [ ]:
df_test['tags'] = preds
df_test

In [ ]:
df_test.to_csv('sample_submission1.csv', index=False)

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))